In [1]:

import numpy as np
import pandas as pd

#读取数据集
full=pd.read_csv("./电信客户流失.csv")
full.head()


,USER_ID,FLOW,FLOW_LAST_ONE,FLOW_LAST_TWO,MONTH_FEE,MONTHS_3AVG,BINDEXP_DATE,PHONE_CHANGE,AGE,OPEN_DATE,REMOVE_TAG
0,83522,155477113,28271889,142960142,80.90,93.77,19000101,0,35,20120117,A
1,83546,319746076,218500270,222357459,99.27,123.27,20171101,0,28,20071001,A
2,83622,123470391,169988672,194224301,130.23,103.35,19000101,0,27,20140221,A
3,83636,282151663,159680673,101173363,160.50,182.17,20180501,0,34,20031031,A
4,83642,380292,1205683,8547991,172.70,174.76,19000101,0,30,20060217,A


In [3]:

'''检查缺失值'''
missingTotal=full.isnull().sum()
missingExist=missingTotal[missingTotal>0]
missingExist=missingExist.sort_values(ascending=False)
print('缺失值计数统计如下：\n',missingExist)


print('删除缺失值前行数为：',full.shape[0])

full=full.dropna(subset=['FLOW_LAST_TWO'],how='any')

print('删除缺失值后行数为：',full.shape[0])


删除缺失值前行数为： 112
删除缺失值后行数为： 112


In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:

ChurnCount=full.groupby('REMOVE_TAG')['REMOVE_TAG'].count()
print('\n状态统计：\n',ChurnCount)


客户是否流失分类统计：
 REMOVE_TAG
A    79
B     4
C     3
D     5
G     2
H     1
I     1
J     3
K     9
L     4
Z     1
Name: REMOVE_TAG, dtype: int64


In [8]:

Churn_mapDict={'A':1,'B':0,'C':0,'D':0,'I':0,'K':0,'L':0,'Z':0,'G':0,'H':0,'J':0}
full['REMOVE_TAG']=full['REMOVE_TAG'].map(Churn_mapDict)
full


,USER_ID,FLOW,FLOW_LAST_ONE,FLOW_LAST_TWO,MONTH_FEE,MONTHS_3AVG,BINDEXP_DATE,PHONE_CHANGE,AGE,OPEN_DATE,REMOVE_TAG
0,83522,155477113,28271889,142960142,80.90,93.77,19000101,0,35,20120117,1
1,83546,319746076,218500270,222357459,99.27,123.27,20171101,0,28,20071001,1
2,83622,123470391,169988672,194224301,130.23,103.35,19000101,0,27,20140221,1
3,83636,282151663,159680673,101173363,160.50,182.17,20180501,0,34,20031031,1
4,83642,380292,1205683,8547991,172.70,174.76,19000101,0,30,20060217,1
...,...,...,...,...,...,...,...,...,...,...,...
107,102388,634257795,908763285,1088058723,151.91,132.46,19000101,0,19,20120608,0
108,155083,994359492,557874334,650769283,197.50,198.89,20180601,0,51,20060517,0
109,123633,721292057,571760009,307956465,108.16,111.26,19000101,0,25,20100903,0
110,244792,180093799,108213590,677305473,106.54,130.26,20170101,0,46,20090713,0


In [9]:
full_X=pd.concat([
                  full['FLOW'],
                  full['FLOW_LAST_ONE'],
                  full['FLOW_LAST_TWO'],
                  full['MONTH_FEE'],
                  full['MONTHS_3AVG'],
                  ],axis=1)

In [10]:
source_X=full_X

source_y=full.loc[:,'REMOVE_TAG']   
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(source_X,source_y,train_size=0.8,test_size=0.2)
print('\n原始数据特征：',source_X.shape,
      '，训练数据特征：',X_train.shape,
      '，测试数据特征：',X_test.shape)
print('原始数据标签：',source_y.shape,
      '，训练数据标签：',y_train.shape,
      '，测试数据标签：',y_test.shape)


原始数据特征： (112, 5) ，训练数据特征： (89, 5) ，测试数据特征： (23, 5)
原始数据标签： (112,) ，训练数据标签： (89,) ，测试数据标签： (23,)


In [12]:
model=LogisticRegression(solver='liblinear')


#训练模型
model.fit(X_train,y_train)

score=model.score(X_test,y_test)
print('\nACCURACY：',score)


ACCURACY： 0.7391304347826086
